In [106]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import sys
import time
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import matplotlib as mpl

import tensorflow as tf
from tensorflow import keras


torch.cuda.is_available()

True

In [107]:
x = torch.rand(5, 3)
print(x)

tensor([[0.3776, 0.4267, 0.9828],
        [0.4513, 0.7101, 0.9351],
        [0.1465, 0.6348, 0.0253],
        [0.2928, 0.3162, 0.2430],
        [0.7008, 0.7776, 0.3549]])


In [108]:

import os.path


train_all = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

NUMERIC_FEATURE_NAMES = [
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
]

CATEGORICAL_FEATURE_NAMES = ["HomePlanet","Destination"]


train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [109]:
test_data_file = "test_data.csv"
test.to_csv(test_data_file, index=False, header=True)

train_all["CryoSleep"].replace([True,False], ["1", "0"], inplace=True)
train_all["VIP"].replace([True,False], ["1", "0"], inplace=True)
test["CryoSleep"].replace([True,False], ["1", "0"], inplace=True)
test["VIP"].replace([True,False], ["1", "0"], inplace=True)

train_all["Relation"] = train_all["PassengerId"].str.split("_").str[1].astype(float)
test["Relation"] = test["PassengerId"].str.split("_").str[1].astype(float)

# train_all["Deck"] = train_all["Cabin"].str[0]
# test["Deck"] = test["Cabin"].str[0]

# train_all["Side"] = train_all["Cabin"].str[2]
# test["Side"] = test["Cabin"].str[2]


train_all.drop(['PassengerId', 'Cabin','Name'],axis =1 , inplace=True)
test.drop(['PassengerId', 'Cabin','Name'],axis = 1 ,inplace=True)
train_all.Transported.replace([True,False], ["1", "0"], inplace=True)

In [110]:

train_all["Age"].fillna(train_all["Age"].mean(skipna=True), inplace=True)
test["Age"].fillna(test["Age"].mean(skipna=True), inplace=True)

train_all["RoomService"].fillna(train_all["RoomService"].mean(skipna=True), inplace=True)
test["RoomService"].fillna(test["RoomService"].mean(skipna=True), inplace=True)

train_all["FoodCourt"].fillna(train_all["FoodCourt"].mean(skipna=True), inplace=True)
test["FoodCourt"].fillna(test["FoodCourt"].mean(skipna=True), inplace=True)

train_all["ShoppingMall"].fillna(train_all["ShoppingMall"].mean(skipna=True), inplace=True)
test["ShoppingMall"].fillna(test["ShoppingMall"].mean(skipna=True), inplace=True)

train_all["Spa"].fillna(train_all["Spa"].mean(skipna=True), inplace=True)
test["Spa"].fillna(test["Spa"].mean(skipna=True), inplace=True)

train_all["VRDeck"].fillna(train_all["VRDeck"].mean(skipna=True), inplace=True)
test["VRDeck"].fillna(test["VRDeck"].mean(skipna=True), inplace=True)

#  ["HomePlanet","CryoSleep","Destination","VIP"]

train_all["HomePlanet"].fillna('Kepler', inplace=True)
test["HomePlanet"].fillna('Kepler', inplace=True)

train_all["CryoSleep"].fillna('0', inplace=True)
test["CryoSleep"].fillna('0', inplace=True)

train_all["VIP"].fillna('0', inplace=True)
test["VIP"].fillna('0', inplace=True)

train_all["Destination"].fillna('TRAPPIST-1e', inplace=True)
test["Destination"].fillna('TRAPPIST-1e', inplace=True)

train_all.dropna(inplace = True)
# test.dropna(inplace = True)

In [111]:
planets = train_all["HomePlanet"].unique()
indices = tf.range(len(planets), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(planets, indices)

table = tf.lookup.StaticVocabularyTable(table_init, 1)

In [112]:
aaa = tf.constant(train_all["HomePlanet"].values)
cat_indices = table.lookup(aaa)
cat_one_hot = tf.one_hot(cat_indices, 5)

cat_one_hot


<tf.Tensor: shape=(8693, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)>

In [113]:

train_all = pd.get_dummies(train_all, columns=["HomePlanet","Destination"])
test = pd.get_dummies(test, columns=["HomePlanet","Destination"])

# for planet in planets:
#   train_all[planet].loc[train_all["HomePlanet"] == planet] = 1
#   train_all[planet].loc[train_all["HomePlanet"] != planet] = 0

train_all.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Relation,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Kepler,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0,0,1,0,0,0,0,1
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,1.0,1,0,0,0,0,0,1
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,1.0,0,1,0,0,0,0,1
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,2.0,0,1,0,0,0,0,1
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,1.0,1,0,0,0,0,0,1


In [114]:
# CSV_HEADER = [
#     "HomePlanet",
#     "Destination",
#     "Age",
#     "RoomService",
#     "FoodCourt",
#     "ShoppingMall",
#     "Spa",
#     "VRDeck",
#     "Transported",
#     ]

train_data_file = "train_data.csv"


In [115]:
# def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
#     dataset = tf.data.experimental.make_csv_dataset(
#         csv_file_path,
#         batch_size=batch_size,
#         column_names=CSV_HEADER,
#         label_name="Transported",
#         num_epochs=1,
#         shuffle=shuffle
#     )
#     return dataset.cache()

In [116]:
# train_all = pd.read_csv('./train_data.csv')
# test = pd.read_csv('./test_data.csv')

train, valid = train_test_split(train_all, test_size=0.05, random_state=42, shuffle=True)

train_X, train_y = train.drop('Transported', axis=1), train['Transported']
valid_X, valid_y = valid.drop('Transported', axis=1), valid['Transported']

train_X.to_csv("./train_x.csv", index=False, header=False)
train_y.to_csv("./train_y.csv", index=False, header=False)
train_X.to_csv("./valid_x.csv", index=False, header=False)
train_y.to_csv("./valid_y.csv", index=False, header=False)

test.to_csv("./test_v.csv", index=False, header=False)


train_X, train_y = pd.read_csv('./train_x.csv'), pd.read_csv('./train_y.csv')
valid_X, valid_y = pd.read_csv('./valid_x.csv'), pd.read_csv('./valid_y.csv')

test_v = pd.read_csv('./test_v.csv')



train_X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8257 entries, 0 to 8256
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       8257 non-null   int64  
 1   31.0    8257 non-null   float64
 2   0.1     8257 non-null   int64  
 3   0.0     8257 non-null   float64
 4   1489.0  8257 non-null   float64
 5   68.0    8257 non-null   float64
 6   6.0     8257 non-null   float64
 7   0.0.1   8257 non-null   float64
 8   2.0     8257 non-null   float64
 9   0.2     8257 non-null   int64  
 10  0.3     8257 non-null   int64  
 11  1       8257 non-null   int64  
 12  0.4     8257 non-null   int64  
 13  0.5     8257 non-null   int64  
 14  0.6     8257 non-null   int64  
 15  1.1     8257 non-null   int64  
dtypes: float64(7), int64(9)
memory usage: 1.0 MB


In [117]:
test_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4276 entries, 0 to 4275
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       4276 non-null   int64  
 1   27.0    4276 non-null   float64
 2   0       4276 non-null   int64  
 3   0.0     4276 non-null   float64
 4   0.0.1   4276 non-null   float64
 5   0.0.2   4276 non-null   float64
 6   0.0.3   4276 non-null   float64
 7   0.0.4   4276 non-null   float64
 8   1.0     4276 non-null   float64
 9   1.1     4276 non-null   int64  
 10  0.1     4276 non-null   int64  
 11  0.2     4276 non-null   int64  
 12  0.3     4276 non-null   int64  
 13  0.4     4276 non-null   int64  
 14  0.5     4276 non-null   int64  
 15  1.2     4276 non-null   int64  
dtypes: float64(7), int64(9)
memory usage: 534.6 KB


In [118]:

# train_X["relation"] = train_X["PassengerId"].str.split("_").str[1].astype(int)
# train_X["relation"]

In [119]:
# train_X["Deck"] = train_X["Cabin"].str[0]

# train_X["Side"] = train_X["Cabin"].str[2]

# train_X.drop(["Cabin", "Name"], axis=1, inplace=True)

# cat_encoder = OneHotEncoder()
# housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Deck"]])
# housing_cat_encoded.toarray()
# train_X["Deck"] = housing_cat_encoded.toarray()

# cat_encoder = OneHotEncoder()
# housing_cat_encoded  = cat_encoder.fit_transform(train_X[["Side"]])
# housing_cat_encoded.toarray()
# train_X["Side"] = housing_cat_encoded.toarray()

# train_X["Deck"]
# train_X["Side"]

In [120]:
test = np.asarray(test).astype('float32')
test

array([[ 1.      , 27.      ,  0.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 0.      , 19.      ,  0.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 1.      , 31.      ,  0.      , ...,  1.      ,  0.      ,
         0.      ],
       ...,
       [ 1.      , 28.658146,  0.      , ...,  1.      ,  0.      ,
         0.      ],
       [ 0.      , 28.658146,  0.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 1.      , 43.      ,  0.      , ...,  0.      ,  1.      ,
         0.      ]], dtype=float32)

In [127]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.Sequential([
    keras.layers.Dense(16, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dense(64, activation='elu'),
    keras.layers.Dense(128, activation='elu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [128]:
model.fit(train_X, train_y, epochs=20, validation_data=(valid_X, valid_y))

Epoch 1/20
259/259 [==============================] - 3s 9ms/step - loss: 0.5836 - binary_accuracy: 0.7064 - val_loss: 0.5442 - val_binary_accuracy: 0.7540
Epoch 2/20
259/259 [==============================] - 3s 10ms/step - loss: 0.5380 - binary_accuracy: 0.7525 - val_loss: 0.5284 - val_binary_accuracy: 0.7652
Epoch 3/20
259/259 [==============================] - 2s 8ms/step - loss: 0.5303 - binary_accuracy: 0.7598 - val_loss: 0.5235 - val_binary_accuracy: 0.7705
Epoch 4/20
259/259 [==============================] - 2s 7ms/step - loss: 0.5218 - binary_accuracy: 0.7618 - val_loss: 0.5181 - val_binary_accuracy: 0.7738
Epoch 5/20
259/259 [==============================] - 2s 7ms/step - loss: 0.5135 - binary_accuracy: 0.7661 - val_loss: 0.5143 - val_binary_accuracy: 0.7750
Epoch 6/20
259/259 [==============================] - 2s 7ms/step - loss: 0.5110 - binary_accuracy: 0.7701 - val_loss: 0.5106 - val_binary_accuracy: 0.7798
Epoch 7/20
259/259 [==============================] - 2s 7ms/st

In [123]:
y_pred = model.predict(test)
y_pred

array([[0.63855064],
       [0.06760568],
       [0.9801707 ],
       ...,
       [0.9476987 ],
       [0.8592263 ],
       [0.6098144 ]], dtype=float32)

In [124]:
prediction = pd.DataFrame(y_pred, columns=['Transported'])

prediction["PassengerId"] = pd.read_csv('./test_data.csv')["PassengerId"]

prediction["Transported"].loc[prediction['Transported'] > 0.5] = True
prediction["Transported"].loc[prediction['Transported'] <= 0.5] = False

prediction.to_csv("./submit.csv", index=False, header=True)


# .to_csv('submit.csv')
# prediction

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
